In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [5]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [6]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [7]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [8]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [9]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [10]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [11]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [12]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [13]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [14]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [16]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502034


In [17]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad
12054403,1112365284,Bm5bmeE,1,MASC,21.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,22.0,NaN,NaN
12796942,1112224374,ZDPbBmY,0,MASC,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,89.0,NaN,NaN
4078381,1112444252,Ezep99N,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
8201952,1112312932,6r5w8E4,0,FEM,23.0,-,-,-,-,-,...,False,True,False,False,False,True,False,27.0,NaN,NaN
15193946,1111787924,DrJ0kmA,0,MASC,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,132.0,NaN,NaN
8275865,1112265025,zv4BG2b,1,MASC,24.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,15.0,NaN,NaN
10808724,1112248694,LNKao5q,0,MASC,23.0,-,-,-,-,-,...,False,False,False,False,False,False,False,1.0,NaN,NaN
5143496,1112249093,5mq3Q5z,1,FEM,19.0,-,-,-,-,Graduado,...,True,False,False,False,False,True,False,121.0,NaN,NaN
4246301,1112460892,pz9LLY3,0,MASC,29.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,29.0,NaN,NaN
8852817,1112257536,EKdKz8,1,FEM,43.0,-,-,En Curso,-,Graduado,...,False,False,False,False,False,True,False,NaN,NaN,NaN


In [18]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
1672564,1112419038,jk5pbzr,1,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,31.0,0.0,0.000000,False
12902602,1111931601,bOzQlGb,0,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,1.0,0.0,0.000000,False
11958208,1112230196,MVd5Aex,0,MASC,41.0,-,-,-,-,Abandonado,...,False,False,False,False,False,False,2.0,0.0,0.000000,False
8671354,1112382258,eJWD59,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,126.0,0.0,0.000000,False
11235275,1112246784,ekO6Wbr,0,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
4231151,1112362565,8VAvLW,1,MASC,32.0,-,-,-,-,-,...,False,False,False,False,False,False,1299.0,2.0,0.001540,True
5229150,1112084995,aGPrp9,0,MASC,45.0,-,-,-,-,-,...,False,False,False,False,True,False,24.0,0.0,0.000000,False
9555560,1112393693,A3X11Mj,1,MASC,200.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,2.0,0.0,0.000000,False
9037221,1112079850,PmG4P3a,0,MASC,29.0,-,-,-,-,-,...,False,False,False,False,True,False,18.0,0.0,0.000000,False
13857351,1112494488,DrJOxkL,0,MASC,28.0,-,-,-,-,-,...,False,False,False,False,False,False,9.0,0.0,0.000000,False


In [19]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Almacén / Depósito / Expedición,1
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [20]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
2,0z5Dmrd,Junior,4
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,16


In [21]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,22
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,25
3,0z5JW1r,Teletrabajo,1
4,0z5VvGv,Full-time,54


In [22]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
7893083,1112467497,Eze4EXz,0,FEM,25.0,-,-,-,-,Graduado,...,77.0,0.0,0.000000,False,0,17,4,3.0,44.0,44.0
12081947,1112373831,969Wqd5,1,MASC,33.0,-,-,-,-,Graduado,...,26.0,0.0,0.000000,False,4,24,4,18.0,24.0,24.0
1248102,1112295763,8M2NQVR,0,MASC,21.0,-,-,Graduado,-,Graduado,...,38.0,0.0,0.000000,False,0,23,16,0.0,21.0,21.0
8912685,1112411627,akxkGrq,0,MASC,33.0,-,-,-,-,-,...,43.0,0.0,0.000000,False,1,21,17,0.0,34.0,34.0
12638749,1112328929,ZDPoxJb,1,MASC,25.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,5,33,4,0.0,0.0,0.0
3073869,1112378542,JBeojok,1,MASC,64.0,-,Graduado,-,-,Graduado,...,6.0,0.0,0.000000,False,3,31,1,0.0,6.0,6.0
4818956,1112341591,bOABLjB,0,FEM,27.0,-,-,Graduado,-,Graduado,...,0.0,0.0,0.000000,False,0,15,11,0.0,0.0,0.0
3713027,1112219991,mzb5Zo9,0,MASC,26.0,-,-,-,-,Graduado,...,19.0,0.0,0.000000,False,1,88,22,2.0,17.0,17.0
8785900,1111848493,JBxQAMO,0,FEM,20.0,-,-,-,-,-,...,0.0,0.0,0.000000,False,1,6,3,0.0,0.0,0.0
9725815,1112454264,OqmBl5r,0,MASC,28.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,3,28,17,0.0,0.0,0.0


In [23]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  15281900 Test:  804311


In [24]:
gc.collect()

312

In [ ]:
c=lgb.LGBMClassifier(learning_rate=0.05,objective='binary',num_leaves=12000,max_depth=18,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.658312
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.627033
[3]	valid_0's binary_logloss: 0.597556
[4]	valid_0's binary_logloss: 0.569665
[5]	valid_0's binary_logloss: 0.544345
[6]	valid_0's binary_logloss: 0.52181
[7]	valid_0's binary_logloss: 0.500921
[8]	valid_0's binary_logloss: 0.480833
[9]	valid_0's binary_logloss: 0.463009
[10]	valid_0's binary_logloss: 0.446528
[11]	valid_0's binary_logloss: 0.430401
[12]	valid_0's binary_logloss: 0.415403
[13]	valid_0's binary_logloss: 0.401543
[14]	valid_0's binary_logloss: 0.390044
[15]	valid_0's binary_logloss: 0.378887
[16]	valid_0's binary_logloss: 0.369044
[17]	valid_0's binary_logloss: 0.358196
[18]	valid_0's binary_logloss: 0.349108
[19]	valid_0's binary_logloss: 0.339544
[20]	valid_0's binary_logloss: 0.331544
[21]	valid_0's binary_logloss: 0.323127
[22]	valid_0's binary_logloss: 0.315192
[23]	valid_0's binary_logloss: 0.308528
[24]	valid_0's binary_logloss

In [26]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9825216059822158


In [27]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV27.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV27.pkl']

In [43]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['lo_vio']=(prediccion['cantidad']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
6428,84527,1112466576,3Nz0Qza,FEM,24.0,-,-,Graduado,-,Graduado,...,292.0,1.0,0.003425,True,24.0,51.0,19.0,84.0,183.0,183.0
28595,29476,1112372029,zv46m0E,FEM,26.0,-,Abandonado,-,Graduado,Graduado,...,NaN,NaN,0.000000,False,0.0,16.0,13.0,0.0,0.0,0.0
9322,86189,1112466706,2zQxNKw,MASC,32.0,-,-,-,-,Graduado,...,NaN,NaN,0.000000,False,0.0,17.0,14.0,0.0,0.0,0.0
56154,5764,1111587152,aBBzPD,MASC,31.0,-,En Curso,-,Graduado,Graduado,...,10.0,NaN,0.000000,False,0.0,1.0,5.0,0.0,0.0,0.0
34807,56366,1112448621,xkvWw5P,MASC,19.0,-,-,En Curso,-,Graduado,...,334.0,1.0,0.002994,True,2.0,492.0,368.0,1.0,240.0,240.0
5908,43152,1112424668,bOJBQM2,FEM,25.0,-,-,-,-,Graduado,...,51.0,9.0,0.176471,True,1.0,18.0,15.0,33.0,30.0,30.0
2898,74836,1112463897,Drz3k1A,FEM,24.0,-,-,-,-,Graduado,...,61.0,1.0,0.016393,True,49.0,77.0,84.0,25.0,33.0,33.0
73058,19115,1112222867,Nroz4B,FEM,39.0,-,-,-,-,Graduado,...,3.0,NaN,0.000000,False,1.0,19.0,17.0,0.0,3.0,3.0
39547,40640,1112420043,bOjeqDV,FEM,37.0,-,-,-,-,-,...,9.0,2.0,0.222222,True,1.0,17.0,15.0,2.0,7.0,7.0
51310,51909,1112442182,LN8kvm5,FEM,25.0,-,-,Graduado,-,Graduado,...,145.0,3.0,0.020690,True,49.0,102.0,106.0,46.0,98.0,98.0


In [44]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[9.94681439e-01, 5.31856136e-03],
       [9.85462004e-01, 1.45379962e-02],
       [9.95929268e-01, 4.07073218e-03],
       ...,
       [9.99411222e-01, 5.88778267e-04],
       [9.99222528e-01, 7.77472213e-04],
       [9.99763768e-01, 2.36231703e-04]])

In [45]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [46]:
prediccion['sepostulo']=y_final2

In [47]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,NaN,0.0,False,0.0,18.0,0.0,4.0,24.0,24.0,0.005319
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,NaN,0.0,False,5.0,60.0,0.0,0.0,6.0,6.0,0.014538
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,NaN,0.0,False,2.0,19.0,1.0,1.0,7.0,7.0,0.004071
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,NaN,0.0,False,0.0,19.0,0.0,0.0,0.0,0.0,0.000399
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,NaN,0.0,False,0.0,12.0,0.0,0.0,3.0,3.0,0.000061


In [48]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.005319
1,1,0.014538
2,2,0.004071
3,3,0.000399
4,4,0.000061


In [49]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion58.csv",index=False)

In [50]:
prediccion.sepostulo.mean()

0.4575268134795598